In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
oil_df=pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/oil.csv")
sample_submission_df=pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv")
holiday_events_df=pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv")
stores_df=pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/stores.csv")
train_df=pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/train.csv")
test_df=pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/test.csv")
transactions_df=pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/transactions.csv")

In [ ]:
"""
Daily oil price. Includes values during both the train and test data timeframes.
(Ecuador is an oil-dependent country and it's economical health is highly vulnerable
to shocks in oil prices.)

Take into account that the stock market does not work on weekends. Therefore we will
have jumps in days that do not correspond to datapoints directly, i.e., the price
list with 1000 prices does not correspond to 1000 prices in row days, it is
the price of the first 1000 workdays (discarding holidays, festivities and weekends)
"""
print("This pandas is oil_df\n")
print("Size of the pandas is: ", oil_df.shape)
print(" ")
print("The number of null elements (NaN) is: \n", oil_df.isnull().sum())
print(" ")
#It can be observed the only missing data is in oil_df
print("Information is: \n", oil_df.describe())
print(" ")


#Let´s add two columns for our interest:
oil_df["Month"]=oil_df["date"].apply(lambda x: int(x.split("-")[1]))
oil_df["Year"]=oil_df["date"].apply(lambda x: int(x.split("-")[0].strip()))
oil_df.head()

In [ ]:
"""
Observe empty areas, that is the NaN part, we will make a tendency to make those missing data be filled
We will fill this data too with an RNN, training with the first not NaN and then filling the NaN part.
We will divide it in sections
"""

# xtick_positions=[0, 1*365, 2*365, 3*365, 4*365] #We can´t do this because of what we explained about weekdays and holidays before
xtick_positions=[oil_df.loc[oil_df['Year'] == 2013].index[0], oil_df.loc[oil_df['Year'] == 2014].index[0],
                 oil_df.loc[oil_df['Year'] == 2015].index[0], oil_df.loc[oil_df['Year'] == 2016].index[0],
                 oil_df.loc[oil_df['Year'] == 2017].index[0]]
xtick_labels=["2013", "2014", "2015", "2016", "2017"]

plt.plot(oil_df["date"], oil_df["dcoilwtico"], label="Oil price")
plt.title("Oil price per day in Ecuador (2013-2017)")
plt.xlabel("Year")
plt.ylabel("Oil price")
plt.xticks(xtick_positions, xtick_labels)
plt.legend()
plt.grid("True")

In [ ]:
"""
Checking both plots, we can corroborate that the years that had the highest oil price were 2013 and 2014.
After year 2014 the price plumbed. Something happened.
"""


dcoilwtico_hist=pd.concat([oil_df[oil_df.Year==2013]['dcoilwtico'],oil_df[oil_df.Year==2014]['dcoilwtico'], oil_df[oil_df.Year==2015]['dcoilwtico'], oil_df[oil_df.Year==2016]['dcoilwtico'], oil_df[oil_df.Year==2017]['dcoilwtico']],axis=1)
dcoilwtico_hist.columns=["2013", "2014", "2015", "2016", "2017"]
dcoilwtico_hist.plot(kind="hist", bins=30, edgecolor="black", figsize=(7,5), alpha=0.35)
plt.xlabel("dcoilwtico")
plt.grid("True")
plt.legend(ncol=3)
plt.title("dcoilwtico Frequency per year")

In [ ]:
print("This pandas is holiday_evets_df\n")
print("Size of the pandas is: ", holiday_events_df.shape)
print(" ")
print("The number of null elements (NaN) is: \n", holiday_events_df.isnull().sum())
print(" ")
print("Information is: \n", holiday_events_df.describe())
print(" ")


#Let´s add two columns for our interest:
holiday_events_df["Month"]=holiday_events_df["date"].apply(lambda x: int(x.split("-")[1]))
holiday_events_df["Year"]=holiday_events_df["date"].apply(lambda x: int(x.split("-")[0].strip()))
holiday_events_df.head()

In [ ]:
# holiday_events_df["description"].unique() # Here we found that there was a worldwide futbol competition and an earthquake
# Let´s check the dates

longest_events_dates_df=pd.concat([holiday_events_df[holiday_events_df.description.str.contains("Mundial")], holiday_events_df[holiday_events_df.description.str.contains("Terremoto")]])
#These dates can be of interest. Let´s observe when these two events started:
print("Mundial in Brazil started: {} and Earthquake stated: {}".format(holiday_events_df.iloc[holiday_events_df[holiday_events_df.description.str.contains("Mundial")].index[0]]["date"],
                                                                      holiday_events_df.iloc[holiday_events_df[holiday_events_df.description.str.contains("Terremoto")].index[0]]["date"]))

In 2014 they were almost at their top in oil price and in 2016 they were almost at the bottom. The earthquake undermined the oil price at that moment with the last dump.

In [ ]:
"""
stores.csv:

Store metadata, including city, state, type, and cluster.
cluster is a grouping of similar stores.
"""
print("This pandas is stores_df\n")
print("Size of the pandas is: ", stores_df.shape)
print(" ")
print("The number of null elements (NaN) is: \n", stores_df.isnull().sum())
print(" ")
print("Information is: \n", stores_df.describe())
print(" ")

stores_df.head()

In [ ]:
"""
train.csv:

The training data, comprising time series of features store_nbr, family, and onpromotion as well as the target sales.

store_nbr: identifies the store at which the products are sold.
family: identifies the type of product sold.
sales: gives the total sales for a product family at a particular store at a given date. Fractional values are possible since products can be sold in fractional units (1.5 kg of cheese, for instance, as opposed to 1 bag of chips).
onpromotion: gives the total number of items in a product family that were being promoted at a store at a given date.
"""

print("This pandas is train_df\n")
print("Size of the pandas is: ", train_df.shape)
print(" ")
print("The number of null elements (NaN) is: \n", train_df.isnull().sum())
print(" ")
print("Information is: \n", train_df.describe())
print(" ")

#Let´s add two columns for our interest:
train_df["Month"]=train_df["date"].apply(lambda x: int(x.split("-")[1]))
train_df["Year"]=train_df["date"].apply(lambda x: int(x.split("-")[0]))
train_df["Day"]=train_df["date"].apply(lambda x: int(x.split("-")[2]))
                                                         
train_df.head()

In [ ]:
xtick_positions=[train_df.loc[train_df['Year'] == 2013].index[0], train_df.loc[train_df['Year'] == 2014].index[0],
                 train_df.loc[train_df['Year'] == 2015].index[0], train_df.loc[train_df['Year'] == 2016].index[0],
                 train_df.loc[train_df['Year'] == 2017].index[0]]
xtick_labels=["2013", "2014", "2015", "2016", "2017"]

plt.figure(figsize=(18,5))
plt.plot(np.arange(len(train_df[(train_df.store_nbr==1) & (train_df.family=="AUTOMOTIVE")].sales)),
         train_df[(train_df.store_nbr==1) & (train_df.family=="AUTOMOTIVE")].sales, label="AUTOMOTIVE nbr = 1")
plt.title("Sales price per day in AUTOMOTIVE nbr=1 (2013-2017)")
plt.xlabel("Year")
plt.ylabel("AUTOMOTIVE sales price")
#plt.xticks(xtick_positions, xtick_labels)
plt.legend()
plt.grid("True")

In [ ]:
nbr_sales=train_df.groupby(["store_nbr"], as_index=False).agg({"sales":"sum"})

plt.figure(figsize=(18,5))
sns.barplot(x=nbr_sales.index, y="sales", data=nbr_sales)
plt.title("Sales per Store")
plt.xlabel("Store")
plt.ylabel("Sales")
#plt.xticks(xtick_positions, xtick_labels)
plt.legend()
plt.grid("True")

In [ ]:
new_year_table=train_df.groupby(["Year"], as_index=False).agg({"sales":"sum"})
#as_index=False to set the Year as a column and have another index instead of Year
plt.bar(new_year_table.Year, new_year_table.sales)
plt.title("Sales per year")
plt.ylabel("Sales")
plt.xlabel("Year")
plt.grid("True")

In [ ]:
nrows=2
ncols=3
fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(16, 8))
plt.subplots_adjust(wspace=0.35, hspace=0.35)
i=0 #i=row
j=0 #j=column
for year in train_df["Year"].unique():
    
    selected_subtable=train_df[train_df["Year"]==year]
    selected_subtable=selected_subtable.groupby(["Month"], as_index=False).agg({"sales":"sum"})
    
    axs[i,j].bar(selected_subtable.Month, selected_subtable.sales)
    axs[i,j].set_xticks(selected_subtable.Month)
    axs[i,j].set_xticklabels(selected_subtable.Month, rotation=50)
    axs[i,j].set_title("Sales per month {}".format(str(year)))
    axs[i,j].set_xlabel("Month")
    axs[i,j].set_ylabel("Sales")
    axs[i,j].grid(True)
    j=j+1
    if j==ncols:
        i=i+1
        j=0

Let´s analyze the best (apparently) year 2015

In [ ]:
store_train_df=train_df.merge(stores_df, how="inner", on="store_nbr")
"""
It follows a very similar style as with SQL. It performs an inner join between two DataFrames
train_df and stores_df based on the common column "store_nbr".

The resulting DataFrame "store_train_df" will contain the combined data from both DataFrames,
where the rows with matching "store_nbr" values are merged together.
"""

#Let´s check the year 2015
store_train_2015_df=store_train_df[store_train_df.Year==2015]
store_train_2015_df.columns

In [ ]:
list_name=["family", "onpromotion", "state", "city", "type", "cluster", "store_nbr"]
plt.subplots_adjust(wspace=0.35, hspace=0.35)
for name in list_name:
    plt.figure(figsize=(15,5))
    sales_train_2015_df=store_train_2015_df.groupby([name], as_index=False).agg({"sales":"sum"})
    sns.barplot(x=sales_train_2015_df[name], y="sales", data=sales_train_2015_df)
    plt.title("Sales per {} year {}".format(name,2015))
    plt.ylabel("Sales")
    plt.xlabel(name)
    if name=="state" or name=="city" or name=="family":
        plt.xticks(rotation=70)
    elif name=="onpromotion":
        plt.xticks(range(0, len(sales_train_2015_df[name]), 10))
    else:
        plt.xticks(rotation=0)
    plt.grid("True")
    plt.show()

In [ ]:
"""
Let´s analyze the transactions_df
Let´s convert the date column to "datetime" to apply ".dt" and obtain day of the week.
"""
transactions_df["date"]=pd.to_datetime(transactions_df["date"])
transactions_df["Day_of_week"]=transactions_df["date"].dt.day_name()
transactions_df["Month"]=transactions_df["date"].dt.month
transactions_df["Year"]=transactions_df["date"].dt.year.astype(str)
transactions_df

In [ ]:
"""
Let´s do the same as before but with two input features to create a heatmap.
"""

list_names=["Month", "Day_of_week"]
for name in list_names:
    plt.figure(figsize=(8,8))
    new_subtable=transactions_df.groupby([name, "Year"], as_index=False).agg({"transactions":"sum"})
    heat_map=new_subtable.pivot(name, "Year", "transactions")
    sns.heatmap(heat_map, annot=True, linecolor="white", linewidths=0.5, cmap="YlGnBu")
    plt.xlabel("Year")
    plt.ylabel(name)
    plt.title("Number of transactions per {} and year".format(name))
    plt.show()

In [ ]:
holiday_train_df=train_df.merge(holiday_events_df, how="inner", on="date")
"""
Performs an inner join between two DataFrames "train_df" and "holiday_events_df" based on the common column "date".
how="inner" specifies that an inner join should be performed, meaning only the matching rows between the two DataFrames
will be included in the resulting DataFrame.
"""
holiday_train_df

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

list_name=["type", "locale", "locale_name", "transferred"]
list_year=list(holiday_train_df.Year_y.unique())
colors=["blue", "red", "green", "yellow", "purple"]

for year in list_year:
    print("Year {}:".format(year))
    holiday_train_year_df = holiday_train_df[holiday_train_df.Year_y == year]

    nrows=2
    ncols=2
    fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(16, 8))

    i=0  # rows
    j=0  # columns

    for name in list_name:
        hist_holiday_train_year_df = holiday_train_year_df.groupby([name], as_index=False).agg({"sales": "sum"})
        axs[i,j].bar(hist_holiday_train_year_df[name], hist_holiday_train_year_df["sales"], color="blue", alpha=0.5, edgecolor="black", linewidth=1.5)
        axs[i,j].set_title("Sales per {} year {}".format(name, year))
        axs[i,j].set_ylabel("Sales")
        axs[i,j].set_xlabel(name)
        if name=="locale_name":
            axs[i,j].tick_params(axis="x", rotation=70)
        else:
            axs[i,j].tick_params(axis="x", rotation=0)
        axs[i, j].grid(True)
        j += 1
        if j == ncols:
            i += 1
            j = 0

    plt.tight_layout()
    plt.show()
    print("\n")
    print("\n")
    print("\n")

# **Feature Engineering**

Feature engineering is the process of creating new features or modifying existing features in a dataset to improve the performance and effectiveness of machine learning models. It involves transforming raw data into a format that is more suitable for the model to learn from.

In [ ]:
from tqdm import tqdm

In [ ]:

plt.hist(train_df.sales, bins=60)
plt.grid(True)
plt.show()
print("The data is too skewed, let´s apply log transformation to reduce the skewness.")



logsale=[]
for sale in tqdm(train_df.sales):
    sale=np.log1p(float(sale))
    logsale.append(sale)

train_df["logsale"]=logsale

plt.hist(train_df.logsale, bins=30)
plt.grid(True)
plt.show()

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
encoder=LabelEncoder()
train_df["family"]=encoder.fit_transform(train_df["family"].values)
train_df

In [ ]:
label_name=["family", "city", "state", "type"]
for name in label_name:
    store_train_df[name]=encoder.fit_transform(store_train_df[name].values)
store_train_df

In [ ]:
test_df

In [ ]:
train_df

In [ ]:
print("Size of the pandas is: ", test_df.shape)
print(" ")
print("The number of null elements (NaN) is: \n", test_df.isnull().sum())
print(" ")
print("Information is: \n", test_df.describe())
print(" ")
test_df.head()
#We have to predict sales. We will use a RNN (LSTM)

In [ ]:
print("Size of the pandas is: ", transactions_df.shape)
print(" ")
print("The number of null elements (NaN) is: \n", transactions_df.isnull().sum())
print(" ")
print("Information is: \n", transactions_df.describe())
print(" ")
transactions_df.head()

**Building the RNN**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN

In [ ]:
#Build the RNN architecture
model=Sequential()
model.add(SimpleRNN(units=32, input_shape(10,1)))
model.add(Dense(units=1))

#Compile model
model.compile(optimizer="adam", loss="mean_squared_error")

#Train the model
model.fit(X,y, epochs=10, batch_size=32)